# Working with Binary Search Trees

## [1/12] Introduction to AVL Trees

In the "Binary Trees" notebook, we implemented binary search trees. A binary search tree is a binary tree wherein the values on the left subtree are smaller than the node value, and the values on the right subtree are bigger than the node value:


<center>
<img src="https://drive.google.com/uc?id=17_NJQM4KCoIaDSln31ZNfkyudbit6dNo" width="30%">
</center>

We learned that the time complexity of the tree methods depends on the height of the binary search tree, which, in turn, depends on the order in which the values are added.

In the worst case, the height will be proportional to the number of values in the tree, making it no better than a list. In the best case, it can be proportional to the logarithm of the number of values. For some perspective, with one million values, the best-case height is only about 20. In this case, we only need to look at 20 nodes to look up a value, even though there are one million values.


The goal of this lesson is to change the tree implementation so that its height is always logarithmic, no matter the order in which values are inserted.

More precisely, we'll implement [AVL Trees](https://en.wikipedia.org/wiki/AVL_tree), which are self-balancing binary search trees. The name "AVL" comes from the initials of this data structure's inventors.

We will base our implementation on the **``Node``** and **``BST``** classes of the "Binary Trees" lesson. In Python, we can add new functionality to a class by **extending** it. To extend a class **``ClassA``** with another class **``ClassB``** we need to declare **``ClassB``** using the following syntax def **``ClassB(ClassA)``**:.

Let's start by extending the **``Node``** class with a new class named **``AVLNode``**. A node in an AVL tree needs the same information as a node in a BST plus two other values:

- The height of the subtree rooted at that node.
- The imbalance of that node.

We'll return to the meaning of these values later. In the initialization method of **``AVLNode``**, we need to call the initialization method in **``Node``** to ensure that the **``self.value``** and **``self.children``** attributes are initialized. We can do this using the **``super()``** function, like so:

```python
class AVLNode(Node):

    def __init__(self, value):
        # Initialize the Node attributes
        super().__init__(value)
        # Do initialization stuff specific to the AVLNode
        # ...
```

Let's complete this implementation by initializing the two new parameters mentioned above.

### Instructions

The **``Node``** and **``BST``** classes from the previous lesson are available in file **``bst.py``**. We've already imported them for you. These will remain imported throughout this lesson.

1. Define a class **``AVLNode``** that extends the Node class.
2. Inside the **``AVLNode``** class, define the **``__init__()``** method with two arguments:
  - **``self``**: the self-reference that is automatically passed
  - **``value``**: the value that the node will store
3. Implement the **``__init__()``** method:
  - Use the **``super()``** function to call the **``__init__()``** method from the **``Node``** class providing the value as argument.
  - This initializes a parameter **``self.height``** to **``1``**.
  - This also initializes a parameter **``self.imbalance``** to **``0``**.

In [ ]:
# solved exercise

class Node:
    """
    A class representing a node in a Binary Search Tree (BST).

    Each node contains a value and references to its left and right children.
    """

    def __init__(self, value):
        """
        Initializes a Node with a given value.

        Parameters:
        - value: The value to be stored in the node.

        Both left and right child references are initialized as None.
        """
        self.value = value
        self.left_child = None
        self.right_child = None


class AVLNode(Node):
    """
    Represents a node in an AVL (Adelson-Velsky and Landis) tree, which is a self-balancing binary search tree.

    Attributes:
        height (int): Tracks the height of the subtree rooted at this node. Initializes to 1 for a new node.
        imbalance (int): Represents the height difference between the left and right children of this node.
                         It helps in determining when rotations might be needed to rebalance the tree. Initializes to 0.
    Inherits:
        value: The value stored in this node.
        left_child (Node or None): Reference to the left child node. Initializes to None.
        right_child (Node or None): Reference to the right child node. Initializes to None.
    """

    def __init__(self, value):
        """
        Initializes an AVLNode with a given value.

        Args:
            value: The value to be stored in the node.
        """
        super().__init__(value)
        self.height = 1
        self.imbalance = 0


## [2/12] Node Height and Imbalance

On the previous section, we implemented the class that we'll use to represent nodes in an **AVL tree**. These nodes store two extra values:

- The height of the subtree rooted at that node.
- The imbalance of that node.

On this section, we'll explain what they are and how they're calculated.

The definition of the height of a node works the same way as the height of a tree. The difference is that the height of the tree is relative to the root node. The height of a non-root node is the number of nodes in the longest path from that node to a leaf. Here are a few examples:


<center>
<img src="https://drive.google.com/uc?id=1ljg6-t3b_FsiZvHdVQw98uRCLVCzLx3V" width="90%">
</center>

If we know the height of the left and right children of a node, we can calculate the height of that node by adding one to the maximum of these two heights. For example, the height of the left child of **``node 9``** is **``2``** and the height of its right child is **``1``**. Therefore, the height of **``node 9``** is **``1 + max(2, 1) = 1 + 2 = 3``**.

In general,

>$
\displaystyle
\textrm{height(node)} = 1 + max(\textrm{height(node.left_child)}, \textrm{height(node.right_child)})
$

If one (or both) of the children doesn't exist, we consider the height of that child to be zero. For example, we saw that the height of **``node 3``** is **``2``**. This node doesn't have a right child, so we determine that its right child has height 0. Then the formula gives **``1 + max(1, 0) = 1 + 1 = 2``**.

The other value that we're keeping track of is the imbalance. The imbalance of a node is the difference between the height of the left subtree and the right subtree:

>$
\textrm{imbalance(node)} = \textrm{height(node.left_child) - height(node.right_child)}
$

The following diagram shows the imbalance values of all nodes in a tree:


<center>
<img src="https://drive.google.com/uc?id=15UHLrJs1QarYR0wyAQmZM-pHXsxnM061" width="40%">
</center>

Let's implement a method that uses the formulas above to calculate the height and imbalance of a node. This method is going to assume that we know the heights of the left and right children. We'll make sure that this is true later.


### Instructions

We've provided you with the implementation of the **``AVLNode``** class from the previous section.

1. Inside the **``AVLNode class``**, add a method named **``calculate_height_and_imbalance()`** with only the **``self``** argument.
2. Implement the **``calculate_height_and_imbalance()`` method by following these steps:
  - Assign to a variable **``left_height``** the value of **``self.left_child.height``** if **``self.left_child``** isn't **None**. If it is **None**, then set it to **``0``**.
  - Assign to a variable **``right_height``** the value of **``self.right_child.height``** if **``self.right_child``** isn't **None**. If it is **None**, then set it to **``0``**.
  - Set the **``self.height``** value to be **``1``** plus the maximum between **left_height** and **right_height**.
  - Set the **``self.imbalance``** value to be the difference **left_height** and **right_height**.

In [ ]:
# solved exercise

class Node:
    """
    A class representing a node in a Binary Search Tree (BST).

    Each node contains a value and references to its left and right children.
    """

    def __init__(self, value):
        """
        Initializes a Node with a given value.

        Parameters:
        - value: The value to be stored in the node.

        Both left and right child references are initialized as None.
        """
        self.value = value
        self.left_child = None
        self.right_child = None


class AVLNode(Node):
    """
    Represents a node in an AVL (Adelson-Velsky and Landis) tree,
    which is a self-balancing binary search tree.

    Attributes:
        height (int): The height of the subtree rooted at this node,
                      initializes to 1 when the node is created.
        imbalance (int): The imbalance factor of this node, calculated
                         as the difference between the heights of the left
                         and right subtrees. Initializes to 0.

    Inherits from:
        Node: Inherits attributes and methods from the Node class.
    """

    def __init__(self, value):
        """
        Initializes an AVLNode with a given value.

        Args:
            value: The value to be stored in this node.
        """
        super().__init__(value)
        self.height = 1
        self.imbalance = 0

    def calculate_height_and_imbalance(self):
        """
        Calculates the height and imbalance factor of this node based
        on the heights of its left and right children.

        This method assumes that the heights of the children nodes (if they exist)
        are up-to-date.
        """
        # Calculate the height of the left child subtree
        left_height = 0
        if self.left_child is not None:
            left_height = self.left_child.height

        # Calculate the height of the right child subtree
        right_height = 0
        if self.right_child is not None:
            right_height = self.right_child.height

        # Update the height of this node
        self.height = 1 + max(left_height, right_height)

        # Calculate and update the imbalance factor for this node
        self.imbalance = left_height - right_height


## [3/12] Keeping Height and Imbalanced Updated

On the previous section, we gave nodes the ability to calculate their height and imbalance provided that the height of their children is calculated.

The height of a node can only change when we add a new value. Therefore, we need to modify the **``_add_recursive()``** method that we implemented in the "Binary Trees" notebook to ensure that the height and imbalance values remain updated.

On this section, we're going to start implementing the **``AVLTree class``**, which will represent our **AVL tree**. In the same way that the **AVLNode** extends the **Node class**, the **AVLTree class** will extend the **BST class** because most of the functionality is the same.

The constructor of **AVLTree** needs to call the constructor of the **BST** tree to initialize the root. Since the tree doesn't need arguments to be built, we don't pass anything to the constructor.

```python
class AVLTree(BST):

    def __init__(self):
        super().__init__()
```


To keep the height and imbalance of each node updated, we need to change the implementation of the **``_add_recursive()``** method inherited from extending the **``BST class``**.

By defining the **``_add_recursive()**`` method in the **AVLTree class**, we override the implementation inherited from the **BST class**. Any call to this method will execute the new code rather than the old one. Thus we don't need to do anything in the **``add()``** method, which is inherited from the **``BST class``**.

We can keep most of the implementation that we have in the **``BST class``**. The only method that we need to modify is the **``_add_recursive()``** method. Recall that this method is responsible for going down the tree to identify where the value needs to be added and then adding it.

The difference now is that after going left or right down the tree, we need to update the height of the current node. Here's a skeleton of the **AVLTree** implementation:

```python
class AVLTree(BST):

    def __init__(self):
        super().__init__()

    # This code is the same we had in the BST class
    def _add_recursive(self, current_node, value):
        if current_node is None:
            return AVLNode(value)
        if value < current_node.value:
            current_node.left_child = self._add_recursive(current_node.left_child, value)
        else:
            current_node.right_child = self._add_recursive(current_node.right_child, value)
        # We need to update the height and balance of the current node here

        return current_node
```

We need to add code at the end of the **``_add_recursive()``** method to update the height and imbalance value of **``current_node``**. Fortunately, we already implemented a method for this on the previous screen.

We'll also add a new **``get_height()``** method that returns the height of the whole tree.

### Instructions

We've provided you with the partial implementation for the **``AVLTree``** shown in the learn section. Let's finish implementing it by updating the height and imbalance of the current node.

1. Inside the **``_add_recursive()``** method, below the comment and before the return statement, update the height and imbalance for current_node by calling the **``AVLNode.calculate_height_and_imbalance()``** method.
2. Inside the class, after the **``_add_recursive()``** method, define a new **``get_height()``** method with only **``self``** as argument. Implement it so that it returns the height attribute of the root node.
3. Outside of the class, use the **``AVLTree()``** constructor to build an empty AVL tree. Assign it to variable **``avl``**.
4. Use the **``AVLTree.add()``** method (inherited from the BST) to add values 1, 2, and 3 (in that order).
5. Calculate the height of **``avl``** by using the **``AVLTree.get_height()``** method. Assign the result to a variable named **``height``**.

In [ ]:
# solved exercise

class Node:
    """
    A class representing a node in a Binary Search Tree (BST).

    Each node contains a value and references to its left and right children.
    """

    def __init__(self, value):
        """
        Initializes a Node with a given value.

        Parameters:
        - value: The value to be stored in the node.

        Both left and right child references are initialized as None.
        """
        self.value = value
        self.left_child = None
        self.right_child = None


class BST:
    """
    Represents a Binary Search Tree (BST).

    Attributes:
        root (Node or None): The root node of the tree. Initializes to None for an empty tree.
    """

    def __init__(self):
        """Initializes an empty BST."""
        self.root = None

    def add(self, value):
        """
        Inserts a new value into the BST.

        Args:
            value: The value to be added to the tree.

        If the tree is empty, the value becomes the root. Otherwise, the method uses
        a recursive helper function to find the appropriate position to maintain the BST property.
        """
        if self.root is None:
            self.root = Node(value)
        else:
            self._add_recursive(self.root, value)

    def _add_recursive(self, current_node, value):
        """
        Recursively finds the correct position and inserts a value into the BST.

        Args:
            current_node (Node): The node to start the search for the insert position from.
            value: The value to be added to the BST.

        The method determines if the new value should be placed to the left or right of
        the current node. If the target position is empty, the value is inserted.
        Otherwise, the function calls itself recursively with the respective child node.
        """
        if value <= current_node.value:
            if current_node.left_child is None:
                current_node.left_child = Node(value)
            else:
                self._add_recursive(current_node.left_child, value)
        else:
            if current_node.right_child is None:
                current_node.right_child = Node(value)
            else:
                self._add_recursive(current_node.right_child, value)

    def _contains(self, current_node, value):
        """
        Recursively checks if the BST contains the specified value starting from a given node.

        Args:
            current_node (Node): The node to start the search from.
            value: The value to search for in the BST.

        Returns:
            bool: True if the value exists in the subtree rooted at current_node, otherwise False.
        """
        if current_node is None:
            return False
        if current_node.value == value:
            return True
        if value < current_node.value:
            return self._contains(current_node.left_child, value)
        return self._contains(current_node.right_child, value)

    def contains(self, value):
        """
        Checks if the BST contains the specified value.

        Args:
            value: The value to search for in the BST.

        Returns:
            bool: True if the BST contains the value, otherwise False.
        """
        return self._contains(self.root, value)

In [ ]:
# solved exercise

class AVLNode(Node):
    """
    Represents a node in an AVL (Adelson-Velsky and Landis) tree,
    which is a self-balancing binary search tree.

    Attributes:
        height (int): The height of the subtree rooted at this node,
                      initializes to 1 when the node is created.
        imbalance (int): The imbalance factor of this node, calculated
                         as the difference between the heights of the left
                         and right subtrees. Initializes to 0.

    Inherits from:
        Node: Inherits attributes and methods from the Node class.
    """

    def __init__(self, value):
        """
        Initializes an AVLNode with a given value.

        Args:
            value: The value to be stored in this node.
        """
        super().__init__(value)
        self.height = 1
        self.imbalance = 0

    def calculate_height_and_imbalance(self):
        """
        Calculates the height and imbalance factor of this node based
        on the heights of its left and right children.

        This method assumes that the heights of the children nodes (if they exist)
        are up-to-date.
        """
        # Calculate the height of the left child subtree
        left_height = 0
        if self.left_child is not None:
            left_height = self.left_child.height

        # Calculate the height of the right child subtree
        right_height = 0
        if self.right_child is not None:
            right_height = self.right_child.height

        # Update the height of this node
        self.height = 1 + max(left_height, right_height)

        # Calculate and update the imbalance factor for this node
        self.imbalance = left_height - right_height

**Solution**

In [ ]:
class AVLTree(BST):
    """
    Represents an AVL (Adelson-Velsky and Landis) tree, a self-balancing binary search tree.
    Inherits all attributes and methods from the BST class and overrides some to maintain the AVL balance property.

    Attributes:
        Inherits all attributes from the BST class.
    """

    def __init__(self):
        """
        Initializes an empty AVL Tree.
        """
        super().__init__()

    def _add_recursive(self, current_node, value):
        """
        Overrides the BST method to recursively find the correct position and insert a value into the AVL tree.
        Also updates the height and imbalance factor of each node along the path of insertion.

        Args:
            current_node (AVLNode or None): The node to start the search for the insert position from.
            value: The value to be added to the AVL tree.

        Returns:
            AVLNode: The node that gets inserted or the node that was already present in that position.
        """
        if current_node is None:
            return AVLNode(value)

        # Check if current_node is of the base class Node and cast it to AVLNode if necessary
        # This is necessary not to change add() in BST class.
        # When the first node is added, the type of the root is Node, then we need to do a casting
        if isinstance(current_node, Node) and not isinstance(current_node, AVLNode):
          current_node = AVLNode(current_node.value)
          current_node.left_child = self.root.left_child
          current_node.right_child = self.root.right_child
          self.root = current_node

        if value <= current_node.value:
            current_node.left_child = self._add_recursive(current_node.left_child, value)
        else:
            current_node.right_child = self._add_recursive(current_node.right_child, value)

        # Update the height and imbalance factor for the current node
        current_node.calculate_height_and_imbalance()

        return current_node

    def get_height(self):
        """
        Retrieves the height of the AVL Tree.

        Returns:
            int: The height of the tree rooted at self.root. Returns 0 if the tree is empty.
        """
        if self.root is None:
            return 0
        return self.root.height


In [ ]:
# Test the implementation

avl = AVLTree() # Instruction 3
avl.add(1) # Instruction 4
avl.add(2)
avl.add(3)
height = avl.get_height() # Instruction 5
print(height)

## [4/12]  Imbalance and Height Relation

We now have a binary search tree that correctly keeps track of node heights and imbalance. We can use this information to make sure that the tree remains balanced.

To do so, let's first understand the relationship between the height of the tree and the imbalance of a node.

As the name suggests, the imbalance of a node measures the degree of imbalance of the left and right subtrees of a node. If the imbalance is zero, it means that the left and right subtree have the same height. If the imbalance is positive, it means the tree is leaning to the left. If the imbalance is negative, it means that the tree is leaning to the right.

The following figure illustrates this:


<center>
<img src="https://drive.google.com/uc?id=1ppq5TGUGqVaackJf5knYNfJ5mJpCz_95" width="60%">
</center>

We've learned that the overall height of the tree relates directly to how evenly nodes are spread. If all nodes lean on the same side, the tree will be very tall. On the other hand, if nodes are evenly spread on the left and the right, the tree will be much shorter:


<center>
<img src="https://drive.google.com/uc?id=184sdmRLCPH_0yg5rcgMrt0Ig18MKdZj3" width="60%">
</center>

The strategy of AVL trees to maintain a low height is to keep the imbalance factors of all nodes low. This means we will ensure that for any given node, its imbalance is either **``-1, 0 or 1``**. As soon as a node reaches an imbalance value of **``-2 or 2``**, we will apply an operation called a **tree rotation** to restore the balance in the tree.

We'll spend the remainder of this lesson learning about rotation and how to use it to keep the AVL tree balanced.

## [5/12] Left Rotation

To automatically maintain a balanced tree as we insert values, we will implement two methods that rearrange the nodes in the tree.

On this section, we'll implement the first of them, which we call a **left rotation**. The following diagram illustrates the left rotation of **node**:


<center>
<img src="https://drive.google.com/uc?id=1WsLG76lUVzq544bYq72goqs_dfwWNRTi" width="60%">
</center>

Let's break down the diagram. In a left rotation, we call the right child of the rotating node **pivot**. The left rotation makes the rotating **node** become the left child of the **pivot**. This means we need to find a new spot for the existing left child of the **pivot**. We can see from the diagram that the new spot for **``pivot.left``** is as the new right child of **node**.

Let's see if the order properties of the binary search tree still hold after a left rotation:

1. The **``pivot``** value is larger than the **``node``** value because it was the right child of **``node``**. Therefore, **``node``** can be the left child of **``pivot``**.
2. The values in the sub-tree rooted at **``node.left``** are smaller than the **``node``** value and therefore smaller than the **``pivot``** value.
3. The values in the sub-tree rooted at **``pivot.left``** were already on the right of **``node``**. Hence all values in it are bigger than the value stored in **``node``**.
4. The values in the sub-tree rooted at **``pivot.left``** are smaller than the **pivot** value because **``pivot.left``** was the left child of **``pivot``**.


Let's implement a method that performs a left rotation on a node. We'll name the method **``_rotate_left``**. We use an underscore to hint at users of our **AVLTree class** that this method is meant for internal use only. This doesn't prevent them from calling the method, but it helps prevent it.

> Note that when we perform a left rotation, the height and imbalance of **node** and **pivot** might change. Therefore, we need to update them after the rotation.

### Instructions

We've provided you with the implementation of the **``AVLTree class``** from the previous section.

1. Inside the class, define a new **``_rotate_left()``** method with two arguments:
  - **``self``**: the self-reference
  - **``node``**: the node that we want to rotate
2. Implement the **``_rotate_left()``** method by doing the following:
  - Assign the right child of **``node``** to a variable named **``pivot``**.
  - Set the right child of **``node``** to be the left child of **``pivot``**.
  - Set the left child of **``pivot``** to be **``node``**.
  - Update the height and imbalance of the **``node``** by calling the **``AVLNode.calculate_height_and_imbalance()``**.
  - Update the height and imbalance of the **``pivot``** by calling the **``AVLNode.calculate_height_and_imbalance()``**.
  - Return the **``pivot``** node. We will use this return value later.

In [ ]:
# Solved Exercise

class AVLTree(BST):
    """
    Represents an AVL (Adelson-Velsky and Landis) tree, a self-balancing binary search tree.
    Inherits all attributes and methods from the BST class and overrides some to maintain the AVL balance property.

    Attributes:
        Inherits all attributes from the BST class.
    """

    def __init__(self):
        """
        Initializes an empty AVL Tree.
        """
        super().__init__()

    def _add_recursive(self, current_node, value):
        """
        Overrides the BST method to recursively find the correct position and insert a value into the AVL tree.
        Also updates the height and imbalance factor of each node along the path of insertion.

        Args:
            current_node (AVLNode or None): The node to start the search for the insert position from.
            value: The value to be added to the AVL tree.

        Returns:
            AVLNode: The node that gets inserted or the node that was already present in that position.
        """
        if current_node is None:
            return AVLNode(value)

        # Check if current_node is of the base class Node and cast it to AVLNode if necessary
        # This is necessary not to change add() in BST class.
        # When the first node is added, the type of the root is Node, then we need to do a casting
        if isinstance(current_node, Node) and not isinstance(current_node, AVLNode):
          current_node = AVLNode(current_node.value)
          current_node.left_child = self.root.left_child
          current_node.right_child = self.root.right_child
          self.root = current_node

        if value <= current_node.value:
            current_node.left_child = self._add_recursive(current_node.left_child, value)
        else:
            current_node.right_child = self._add_recursive(current_node.right_child, value)

        # Update the height and imbalance factor for the current node
        current_node.calculate_height_and_imbalance()

        return current_node

    def get_height(self):
        """
        Retrieves the height of the AVL Tree.

        Returns:
            int: The height of the tree rooted at self.root. Returns 0 if the tree is empty.
        """
        if self.root is None:
            return 0
        return self.root.height

    def _rotate_left(self, node):
        """
        Performs a left rotation on the given node and adjusts the height and imbalance attributes.

        A left rotation is used to balance an AVL Tree when the right subtree of a node
        becomes higher than the left subtree. The method updates the heights and imbalance
        factors for the rotated nodes.

        Args:
            node (AVLNode): The node to be rotated.

        Returns:
            AVLNode: The new root node of the rotated subtree (the pivot).
        """

        # Store the pivot (the root of the right subtree of 'node')
        pivot = node.right_child

        # Update the right child of 'node' to be the left child of the pivot
        node.right_child = pivot.left_child

        # Set the left child of the pivot to be the node
        pivot.left_child = node

        # Recalculate the height and imbalance factor for the rotated node
        node.calculate_height_and_imbalance()

        # Recalculate the height and imbalance factor for the pivot
        pivot.calculate_height_and_imbalance()

        # Return the pivot as the new root of this subtree
        return pivot


## [6/12] Right Rotaiton

On the previous section, we implemented left rotations. On this section, we'll learn about**``right rotations``**. On the next section, we'll see how these rotations can help maintain binary search tree balance.

The right rotation is analogous to the left rotation. The difference now is that the pivot is the left child of the rotating node, and the rotating node becomes the right child of the pivot:


<center>
<img src="https://drive.google.com/uc?id=17au1PyU-TZELFdKOXxstdm1Gc5ITEupy" width="60%">
</center>

Let's break down the diagram. In the right rotation, we call the left child of the rotating **``node pivot``**. The right rotation consists of making the rotating **``node``** become the right child of the **``pivot``**. This means we need to find a new spot for the existing right child of the **``pivot``**. We can see from the diagram that the new spot for **``pivot.right``** is as the new left child of **``node``**.

Let's see that the order properties of the binary search tree still hold after a right rotation:

1. The **``pivot``** value is smaller than the **``node``** value because it was the left child of **``node``**. Therefore, **``node``** can be the right child of **``pivot``**.
2. The values in the sub-tree rooted at **``node.right``** are larger than the node value and therefore larger than the **``pivot``** value.
3. The values in the sub-tree rooted at **``pivot.right``** were already on the left of **``node``**. Therefore, all values in it are smaller than the value stored in **``node``**.
4. The values in the sub-tree rooted at **``pivot.right``** are larger than the **``pivot``** value because **``pivot.right``** was the right child of **``pivot``**.

Let's implement a method that performs a right rotation on a node. We'll name the method **``_rotate_right``**. In the same way as we did with left rotations, we use an underscore to hint at users of our **``AVLTree``** class that this method is meant for internal use only.

Note that when we perform a right rotation, the height and imbalance of **``node``** and **``pivot``** might change. Therefore, we need to update them after the rotation.

### Instructions

We've provided you with the implementation of the **``AVLTree``** class from the previous section.

1. Inside the class, define a new **``_rotate_right()``** method with two arguments:
  - **``self``**: the self-reference
  - **``node``**: the node that we want to rotate
2. Implement the **``_rotate_right()``** method by:
  - Assign the left child of **``node``** to a variable named **``pivot``**.
  - Set the left child of **``node``** to be the right child of **``pivot``**.
  - Set the right child of **``pivot``** to be **``node``**.
  - Update the height and imbalance of the **``node``** by calling the **``AVLNode.calculate_height_and_imbalance()``**.
  - Update the height and imbalance of the **``pivot``** by calling the **``AVLNode.calculate_height_and_imbalance()``**.
  - Return the **``pivot``** node. We will use this return value later.

In [ ]:
# Solved Exercise

class AVLTree(BST):
    """
    Represents an AVL (Adelson-Velsky and Landis) tree, a self-balancing binary search tree.
    Inherits all attributes and methods from the BST class and overrides some to maintain the AVL balance property.

    Attributes:
        Inherits all attributes from the BST class.
    """

    def __init__(self):
        """
        Initializes an empty AVL Tree.
        """
        super().__init__()

    def _add_recursive(self, current_node, value):
        """
        Overrides the BST method to recursively find the correct position and insert a value into the AVL tree.
        Also updates the height and imbalance factor of each node along the path of insertion.

        Args:
            current_node (AVLNode or None): The node to start the search for the insert position from.
            value: The value to be added to the AVL tree.

        Returns:
            AVLNode: The node that gets inserted or the node that was already present in that position.
        """
        if current_node is None:
            return AVLNode(value)

        # Check if current_node is of the base class Node and cast it to AVLNode if necessary
        # This is necessary not to change add() in BST class.
        # When the first node is added, the type of the root is Node, then we need to do a casting
        if isinstance(current_node, Node) and not isinstance(current_node, AVLNode):
          current_node = AVLNode(current_node.value)
          current_node.left_child = self.root.left_child
          current_node.right_child = self.root.right_child
          self.root = current_node

        if value <= current_node.value:
            current_node.left_child = self._add_recursive(current_node.left_child, value)
        else:
            current_node.right_child = self._add_recursive(current_node.right_child, value)

        # Update the height and imbalance factor for the current node
        current_node.calculate_height_and_imbalance()

        return current_node

    def get_height(self):
        """
        Retrieves the height of the AVL Tree.

        Returns:
            int: The height of the tree rooted at self.root. Returns 0 if the tree is empty.
        """
        if self.root is None:
            return 0
        return self.root.height

    def _rotate_left(self, node):
        """
        Performs a left rotation on the given node and adjusts the height and imbalance attributes.

        A left rotation is used to balance an AVL Tree when the right subtree of a node
        becomes higher than the left subtree. The method updates the heights and imbalance
        factors for the rotated nodes.

        Args:
            node (AVLNode): The node to be rotated.

        Returns:
            AVLNode: The new root node of the rotated subtree (the pivot).
        """

        # Store the pivot (the root of the right subtree of 'node')
        pivot = node.right_child

        # Update the right child of 'node' to be the left child of the pivot
        node.right_child = pivot.left_child

        # Set the left child of the pivot to be the node
        pivot.left_child = node

        # Recalculate the height and imbalance factor for the rotated node
        node.calculate_height_and_imbalance()

        # Recalculate the height and imbalance factor for the pivot
        pivot.calculate_height_and_imbalance()

        # Return the pivot as the new root of this subtree
        return pivot


    def _rotate_right(self, node):
        """
        Performs a right rotation on the given node and adjusts the height and imbalance attributes.

        A right rotation is used to balance an AVL Tree when the left subtree of a node
        becomes higher than the right subtree. This method updates the heights and imbalance
        factors for the rotated nodes.

        Args:
            node (AVLNode): The node around which the rotation will be performed.

        Returns:
            AVLNode: The new root node of the rotated subtree (the pivot).
        """

        # Store the pivot (the root of the left subtree of 'node')
        pivot = node.left_child

        # Update the left child of 'node' to be the right child of the pivot
        node.left_child = pivot.right_child

        # Set the right child of the pivot to be the node
        pivot.right_child = node

        # Recalculate the height and imbalance factor for the rotated node
        node.calculate_height_and_imbalance()

        # Recalculate the height and imbalance factor for the pivot
        pivot.calculate_height_and_imbalance()

        # Return the pivot as the new root of this subtree
        return pivot


## [7/12] Balancing the Tree - Part 1

Great success! We can now perform left and right node rotations. But how exactly does it help us balance the tree?

We've mentioned that our strategy for balancing the tree is to keep the imbalance of a node to **``-1, 0, or 1``**. To do so, when we add a new value to the tree, after we update the height and imbalance of the node, we will check whether it became **``2 or -2``**. In this case, we will rotate the tree to restore the balance.

Let's focus on the case where the imbalance of the current node becomes 2. In this case, it means that the tree is leaning to the left. We need to consider the following two cases, depending on the imbalance of the pivot (note that the diagrams are not showing the full tree):


<center>
<img src="https://drive.google.com/uc?id=16XDSKstnLxvxg3EREDmFPZ-zRCuqzwvm" width="60%">
</center>

In the first case, performing a right rotation on node will result in the following tree:


<center>
<img src="https://drive.google.com/uc?id=1MrtZIOHSFxBgEedM4js4a6a0HEbVBqP0" width="60%">
</center>

Let's see that the imbalances of **``pivot``** and **``node``** are indeed 0 after the rotation. Because of the imbalance factor of **``node``** before rotation, we know the following:

>$
\textrm{height(pivot)} = 2 + \textrm{height(node.right)}
$

Since the imbalance of pivot is **``1``**, we know the following:

>$
\textrm{height(pivot.left)} = 1 + \textrm{height(pivot.right)}
$

Thus, **``pivot.left``** is the highest child of **``pivot``** and, by definition of height, we have the following:

>$
\textrm{height(pivot)} = 1 + \textrm{height(pivot.left)} = 2 + \textrm{height(pivot.right)}
$

By combining this with the first equation, we conclude this:

>$
\textrm{height(pivot.right)} = \textrm{height(node.right)}
$

This shows that the imbalance of **``node``** is **``0``** after the rotation since the heights of **``pivot.right``** and **``node.right``** don't change with rotation.

To show that the imbalance of **``pivot``** is 0, note that by definition of height, after rotation, we have the following:

>$
\textrm{height(node)} = 1 + \textrm{height(pivot.right)}
$

We've already seen this . . .

>$
\textrm{height(pivot.left)} = 1 + \textrm{height(pivot.right)}
$

...so...

>$
\textrm{height(node)} = \textrm{height(pivot.left)}
$

Thus, after rotation, the imbalance of **``pivot``** is **``0``**.

Let's implement this first case where the **``node``** has imbalance equal to 2 and the **``pivot``** has imbalance equal to 1. We'll implement the remaining ones on the following section.


```python
def _balance(self, node):
        if node.imbalance == 2:
            pivot = node.left_child
            if pivot.imbalance == 1:
                return self._rotate_right(node)

...continue
```

## [8/12] Balancing the Tree — Part 2

On the previous section, we started to implement tree balancing. We learned that if the imbalance of a node is 2 and the imbalance of its left child is 1, then a right rotation of the node will restore the tree balance.

On this section, we'll explore the situation when the left child imbalance is -1, as we see in the following diagram:


<center>
<img src="https://drive.google.com/uc?id=1BTqri_c_TxaFDP_GD1Y6N65CoDSDrxkT" width="40%">
</center>

In this case, a single right rotation of node isn't enough, as we see in the following example:


<center>
<img src="https://drive.google.com/uc?id=1uwtXMS5OfJij_JDbs9n6Pgc0X14aeu6_" width="40%">
</center>

We see on the previous image that rotating the **``node``** to the right mirrored the tree horizontally and thus transferred the imbalance from the left to the right.

To fix this, we need to perform an extra rotation. The idea is to first rotate the **``pivot``** to the left. This will make the imbalances of the pivot become 1 instead of -1, bringing us back to the case we solved on the previous screen. Let's apply this to the above example to see if it works:


<center>
<img src="https://drive.google.com/uc?id=1VfII2LInVhe4Ga5ayXtWs7uzcT2fyzig" width="60%">
</center>


In general, when we reach a node with an imbalance that equals **``2``** with a pivot imbalance equal to **``-1``**, we balance the tree by performing these two rotations:

- Rotate the **``pivot``** node to the left.
- Rotate the **``node``** to the right.
Let's continue our implementation from the previous section to account for this case.

```python
def _balance(self, node):
        if node.imbalance == 2:
            pivot = node.left_child
            if pivot.imbalance == 1:
                return self._rotate_right(node)
            # Add else here
            else:
                node.left_child = self._rotate_left(pivot)
                return self._rotate_right(node)
```

## [9/12] Balancing the Tree — Part 3

Most of the work to balance the tree is done! We've implemented the first two of the four imbalance cases — more precisely, the cases where the node imbalance is 2 (in other words, when the tree is leaning to the left):


<center>
<img src="https://drive.google.com/uc?id=1FIDANnv04rDMUEcdiLU3omdl32f8uMV2" width="60%">
</center>

Now we need to implement the final two cases where the node imbalance is -2. Fortunately, these cases are symmetrical. To implement them, we can literally copy the other cases and replace all occurrences of left by right and all occurrences of right by left.


If the imbalance of **``node``** is -2, then the **``pivot``** is the right child of the node. If the **``pivot``** has an imbalance of -1, we can balance the tree by rotating the **``node``** to the left.

If the **``pivot``** has an imbalance of 1, then we need two rotations. First, we rotate the **``pivot``** to the right, and then we rotate the **``node``** to the left.

Here are the rotations necessary in each case (note that lower rotations happen before the top rotation):


<center>
<img src="https://drive.google.com/uc?id=16rBfeGxZAeCqT0AEFn-tOQv9Iq4NHt_F" width="70%">
</center>

We can see that in the last two cases we need to perform the opposite rotations that we did to solve the first two cases.


```python
def _balance(self, node):
        if node.imbalance == 2:
            pivot = node.left_child
            if pivot.imbalance == 1:
                return self._rotate_right(node)
            else:
                node.left_child = self._rotate_left(pivot)
                return self._rotate_right(node)
        # Add elif code here
        elif node.imbalance == -2:
            pivot = node.right_child
            if pivot.imbalance == -1:
                return self._rotate_left(node)
            else:
                node.right_child = self._rotate_right(pivot)
                return self._rotate_left(node)
```


## [10/12] Balancing on Insertion

Congratulations on implementing a method for balancing AVL trees! We now have a fully functional balancing method. All that remains to finish our AVL tree implementation is to call it when inserting new nodes.

We'll call it in the insertion method because this is the only place where the balance of the tree can change. Recall that after going down the tree to insert a new value, we update the height and imbalance of each node on the way up using the **``AVLNode.calculate_height_and_imbalance()``** method.

After calling this method the height and imbalance of the current node will be updated. This is a good moment to restore the tree balance.

We will use **``if``** statement to check whether the current node imbalance is either 2 or -2. If it is, then we will return the result of calling the **``_balance()``** method on the current node instead of returning the current node.

To check if the imbalance is 2 or -2 we could use two conditions. However, we can instead use a single conditions with the **``abs()``** built-in function. This function removes the sign from a number, which we call the **``absolute value``** of the number. For example:

In [ ]:
print(abs(2))
print(abs(-2))

So, we can check whether something equals 2 or -2 by checking whether its absolute value is 2. For instance:

In [ ]:
x = 2
if abs(x) == 2:
    print("It works with 2")

x = -2
if abs(x) == 2:
    print("It also works with -2")

### Instructions [reference code you must use]

We've provided you with the implementation of the **``AVLTree``** class from the previous section.

1. Inside the **``_add_recursive()`` method, after updating the node height and imbalance and before the **``return``** statement, do the following:
  - Add an **``if``** statement that checks whether the current node imbalances equal 2 or -2.
  - If it does, return the result of calling the **``_balance()``** method on the current node.

In [ ]:
# solved exercise

class Node:
    """
    A class representing a node in a Binary Search Tree (BST).

    Each node contains a value and references to its left and right children.
    """

    def __init__(self, value):
        """
        Initializes a Node with a given value.

        Parameters:
        - value: The value to be stored in the node.

        Both left and right child references are initialized as None.
        """
        self.value = value
        self.left_child = None
        self.right_child = None


class BST:
    """
    Represents a Binary Search Tree (BST).

    Attributes:
        root (Node or None): The root node of the tree. Initializes to None for an empty tree.
    """

    def __init__(self):
        """Initializes an empty BST."""
        self.root = None

    def add(self, value):
        """
        Inserts a new value into the BST.

        Args:
            value: The value to be added to the tree.

        If the tree is empty, the value becomes the root. Otherwise, the method uses
        a recursive helper function to find the appropriate position to maintain the BST property.
        """
        if self.root is None:
            self.root = Node(value)
        else:
            self._add_recursive(self.root, value)

    def _add_recursive(self, current_node, value):
        """
        Recursively finds the correct position and inserts a value into the BST.

        Args:
            current_node (Node): The node to start the search for the insert position from.
            value: The value to be added to the BST.

        The method determines if the new value should be placed to the left or right of
        the current node. If the target position is empty, the value is inserted.
        Otherwise, the function calls itself recursively with the respective child node.
        """
        if value <= current_node.value:
            if current_node.left_child is None:
                current_node.left_child = Node(value)
            else:
                self._add_recursive(current_node.left_child, value)
        else:
            if current_node.right_child is None:
                current_node.right_child = Node(value)
            else:
                self._add_recursive(current_node.right_child, value)

    def _contains(self, current_node, value):
        """
        Recursively checks if the BST contains the specified value starting from a given node.

        Args:
            current_node (Node): The node to start the search from.
            value: The value to search for in the BST.

        Returns:
            bool: True if the value exists in the subtree rooted at current_node, otherwise False.
        """
        if current_node is None:
            return False
        if current_node.value == value:
            return True
        if value < current_node.value:
            return self._contains(current_node.left_child, value)
        return self._contains(current_node.right_child, value)

    def contains(self, value):
        """
        Checks if the BST contains the specified value.

        Args:
            value: The value to search for in the BST.

        Returns:
            bool: True if the BST contains the value, otherwise False.
        """
        return self._contains(self.root, value)

In [ ]:
# solved exercise

class AVLNode(Node):
    """
    Represents a node in an AVL (Adelson-Velsky and Landis) tree,
    which is a self-balancing binary search tree.

    Attributes:
        height (int): The height of the subtree rooted at this node,
                      initializes to 1 when the node is created.
        imbalance (int): The imbalance factor of this node, calculated
                         as the difference between the heights of the left
                         and right subtrees. Initializes to 0.

    Inherits from:
        Node: Inherits attributes and methods from the Node class.
    """

    def __init__(self, value):
        """
        Initializes an AVLNode with a given value.

        Args:
            value: The value to be stored in this node.
        """
        super().__init__(value)
        self.height = 1
        self.imbalance = 0

    def calculate_height_and_imbalance(self):
        """
        Calculates the height and imbalance factor of this node based
        on the heights of its left and right children.

        This method assumes that the heights of the children nodes (if they exist)
        are up-to-date.
        """
        # Calculate the height of the left child subtree
        left_height = 0
        if self.left_child is not None:
            left_height = self.left_child.height

        # Calculate the height of the right child subtree
        right_height = 0
        if self.right_child is not None:
            right_height = self.right_child.height

        # Update the height of this node
        self.height = 1 + max(left_height, right_height)

        # Calculate and update the imbalance factor for this node
        self.imbalance = left_height - right_height

**``Solution``**

In [ ]:
# Solved Exercise

class AVLTree(BST):
    """
    Represents an AVL (Adelson-Velsky and Landis) tree, a self-balancing binary search tree.
    Inherits all attributes and methods from the BST class and overrides some to maintain the AVL balance property.

    Attributes:
        Inherits all attributes from the BST class.
    """

    def __init__(self):
        """
        Initializes an empty AVL Tree.
        """
        super().__init__()

    def add(self, value):
        """
        Overrides the add method in the BST class to handle AVL Tree balancing.
        """
        self.root = self._add_recursive(self.root, value)  # Note that self.root is updated here


    def _add_recursive(self, current_node, value):
        """
        Overrides the BST method to recursively find the correct position and insert a value into the AVL tree.
        This method also ensures the tree remains balanced by updating node heights and performing rotations as needed.

        Args:
            current_node (AVLNode or Node or None): The node from which to start the search for the insert position.
            value (Any): The value to be added to the AVL tree.

        Returns:
            AVLNode: The node that either gets inserted or the node that was already present at that position.

        Notes:
            1. The method first checks if the `current_node` is an instance of the base class `Node`.
              If it is, the method casts it to `AVLNode` to ensure AVL properties are maintained. This is especially
              useful if the first node added to the tree is of type `Node`; this ensures subsequent nodes will be of
              type `AVLNode`.
            2. The method also balances the tree by calling the `_balance` method if the imbalance factor
              of a node reaches 2 or -2 after an insert operation.
        """

        # If the current node is None, return a new AVLNode containing the value
        if current_node is None:
            return AVLNode(value)

        # Check if current_node is of the base class Node and cast it to AVLNode if necessary
        # This is necessary to not change the add() in the BST class.
        # When the first node is added, the type of the root is Node, so we need to cast it
        if isinstance(current_node, Node) and not isinstance(current_node, AVLNode):
            current_node = AVLNode(current_node.value)
            current_node.left_child = self.root.left_child
            current_node.right_child = self.root.right_child
            self.root = current_node

        # Determine whether the value should be inserted to the left or right subtree
        if value <= current_node.value:
            current_node.left_child = self._add_recursive(current_node.left_child, value)
        else:
            current_node.right_child = self._add_recursive(current_node.right_child, value)

        # Update the height and imbalance factor for the current node
        current_node.calculate_height_and_imbalance()

        # Check if tree balancing is needed and balance if necessary
        if abs(current_node.imbalance) == 2:
            return self._balance(current_node)

        return current_node

    def get_height(self):
        """
        Retrieves the height of the AVL Tree.

        Returns:
            int: The height of the tree rooted at self.root. Returns 0 if the tree is empty.
        """
        if self.root is None:
            return 0
        return self.root.height

    def _rotate_left(self, node):
        """
        Performs a left rotation on the given node and adjusts the height and imbalance attributes.

        A left rotation is used to balance an AVL Tree when the right subtree of a node
        becomes higher than the left subtree. The method updates the heights and imbalance
        factors for the rotated nodes.

        Args:
            node (AVLNode): The node to be rotated.

        Returns:
            AVLNode: The new root node of the rotated subtree (the pivot).
        """

        # Store the pivot (the root of the right subtree of 'node')
        pivot = node.right_child

        # Update the right child of 'node' to be the left child of the pivot
        node.right_child = pivot.left_child

        # Set the left child of the pivot to be the node
        pivot.left_child = node

        # Recalculate the height and imbalance factor for the rotated node
        node.calculate_height_and_imbalance()

        # Recalculate the height and imbalance factor for the pivot
        pivot.calculate_height_and_imbalance()

        # Return the pivot as the new root of this subtree
        return pivot


    def _rotate_right(self, node):
        """
        Performs a right rotation on the given node and adjusts the height and imbalance attributes.

        A right rotation is used to balance an AVL Tree when the left subtree of a node
        becomes higher than the right subtree. This method updates the heights and imbalance
        factors for the rotated nodes.

        Args:
            node (AVLNode): The node around which the rotation will be performed.

        Returns:
            AVLNode: The new root node of the rotated subtree (the pivot).
        """

        # Store the pivot (the root of the left subtree of 'node')
        pivot = node.left_child

        # Update the left child of 'node' to be the right child of the pivot
        node.left_child = pivot.right_child

        # Set the right child of the pivot to be the node
        pivot.right_child = node

        # Recalculate the height and imbalance factor for the rotated node
        node.calculate_height_and_imbalance()

        # Recalculate the height and imbalance factor for the pivot
        pivot.calculate_height_and_imbalance()

        # Return the pivot as the new root of this subtree
        return pivot

    def _balance(self, node):
      """
      Balances the subtree rooted at the given node by performing rotations as needed.

      If the imbalance factor of the given node is 2 or -2, rotations are performed
      to bring the subtree back into balance. This method also takes into account
      the imbalance factors of the child nodes to decide which type of rotation is needed
      (single or double).

      Args:
          node (AVLNode): The root node of the subtree that needs to be balanced.

      Returns:
          AVLNode: The new root node of the balanced subtree.

      Note:
          This method assumes that the height and imbalance factor of each node are up-to-date.
      """

      # Case 1: Left subtree is higher than right subtree
      if node.imbalance == 2:
          pivot = node.left_child
          # Single right rotation
          if pivot.imbalance == 1:
              return self._rotate_right(node)
          # Double rotation: Left-Right
          else:
              node.left_child = self._rotate_left(pivot)
              return self._rotate_right(node)
      # Case 2: Right subtree is higher than left subtree
      else:
          pivot = node.right_child
          # Single left rotation
          if pivot.imbalance == -1:
              return self._rotate_left(node)
          # Double rotation: Right-Left
          else:
              node.right_child = self._rotate_right(pivot)
              return self._rotate_left(node)

## [11/12] Testing the Implementation

We've completed the AVL tree implementation!

It's a good practice to test our code to ensure that it works correctly. We suggest that you test the **``AVLTree``** implementation on this section.

To encourage you to explore code testing, we don't provide you with explicit instructions on how test it; rather, here is a list some things that you should test:
- We've learned that without rotations, inserting values in sequential order results in an AVL tree with maximum height. Test that rotations are working well by inserting sequential values 1, 2, 3, up to say, 10,000, and checking that the height is much smaller than 10,000.
- Experiment inserting 10,000 random values and checking that the height also remains low in this case. You can use the [random module](https://docs.python.org/3/library/random.html) for this.
- We should also check that the contains method is working correctly. Insert 10,000 random values, and check that they are all contained in the tree afterwards.
- Another thing that would be interesting to check is the execution time of the contains method. Compare using the contains method on an AVL tree with 10,000 values and a list with the same values.

These are just a few ideas on things to test. Feel free to come up with your own ideas.

When testing, the **``assert``** statement can be useful to test a specific condition. The syntax is the following:

```python
assert condition, "optional message"
```

When the code is executed, if **``condition``** is **``False``**, then it will cause an error and display a message (if provided). Here's an example of using an assertion to test the AVL tree:

```python
avl = AVLTree()
avl.add(42)
assert avl.contains(42), "The AVL tree doesn't contain the value 42 after it was added."
```

A more standard way to test code is to use the [unittest module](A more standard way to test code is to use the unittest module; however, this is beyond the scope of this lesson.); however, this is beyond the scope of this lesson.



### Instruction

In [ ]:
# The AVLTree class is available
NUM_VALUES = 100000

In [ ]:
# # Test height with sequential inserts, the height should be much smaller than NUM_VALUES
avl = AVLTree()
for i in range(NUM_VALUES):
    avl.add(i)
print(avl.root.height)

In [ ]:
# Test height with random inserts, the height should be much smaller than NUM_VALUES
import random
random.seed(0)
rnd_values = [random.randint(1, 1000000) for _ in range(NUM_VALUES)]
avl = AVLTree()
for v in rnd_values:
    avl.add(v)
print(avl.root.height)

In [ ]:
# Test contains method
for v in rnd_values:
    assert avl.contains(v)

In [ ]:
import time
import plotly.express as px

# Generate experiment numbers (replace this with your actual experiment numbers)
experiment_numbers = list(range(1, len(rnd_values) + 1))

times_list = []
times_avl = []

for v in rnd_values:
    # Measure runtime for list
    start = time.time()
    v in rnd_values
    end = time.time()
    times_list.append(end - start)

    # Measure runtime for AVL
    start = time.time()
    avl.contains(v)
    end = time.time()
    times_avl.append(end - start)

# Convert to milliseconds for better visualization
times_list_ms = [t * 1000 for t in times_list]
times_avl_ms = [t * 1000 for t in times_avl]

# Create a Plotly figure as a scatter plot

fig = px.scatter(x=[experiment_numbers], y=times_list_ms, labels={"x": "Experiment Number", "y": "Runtime (ms)"},
                 title="Runtime Comparison: List vs. AVL", template="plotly_dark")
fig.add_scatter(x=experiment_numbers, y=times_avl_ms, mode='markers', name='AVL Tree', marker=dict(size=8))

# Update axes properties
fig.update_xaxes(showline=True, linewidth=1, linecolor="gray")
fig.update_yaxes(showline=True, linewidth=1, linecolor="gray")
# Change the name in the legend
fig.update_traces(name="List", selector=dict(name="wide_variable_0"))

# Show the figure
fig.show()


### Data visualization

In [ ]:
import plotly.graph_objects as go
from collections import deque

def plot_avl_tree_iterative(avl_tree):
    if avl_tree.root is None:
        print("The tree is empty.")
        return

    # Use a deque for level order traversal
    node_queue = deque([(avl_tree.root, 0, 0, None)]) # (node, x, y, parent_position)
    nodes = []
    edges = []

    while node_queue:
        current_node, x, y, parent_pos = node_queue.popleft()

        if current_node:
            nodes.append((x, y, current_node.value, current_node.height, current_node.imbalance))
            if parent_pos:
                edges.append((parent_pos, (x, y)))

            # Calculate the horizontal space for the next level
            space = 2 ** (-y - 1)
            node_queue.append((current_node.left_child, x - space, y - 1, (x, y)))
            node_queue.append((current_node.right_child, x + space, y - 1, (x, y)))

    # Create the plotly figure
    fig = go.Figure()

    # Add the edges (lines) to the figure
    for start, end in edges:
        fig.add_trace(go.Scatter(x=[start[0], end[0]], y=[start[1], end[1]], mode='lines', line=dict(color='blue', width=2)))

    # Add the nodes to the figure
    for x, y, value, height, imbalance in nodes:
        fig.add_trace(go.Scatter(x=[x], y=[y], mode='markers+text', text=[f'Value: {value}<br>Height: {height}<br>Imbalance: {imbalance}'], textposition='bottom center', marker=dict(size=10, color='red')))

    # Update layout of the figure
    fig.update_layout(title='AVL Tree Visualization', xaxis=dict(visible=False), yaxis=dict(visible=False), showlegend=False)
    fig.update_xaxes(range=[-1, 1])
    fig.update_yaxes(range=[-len(nodes) // 3 - 1, 1])

    fig.show()


In [ ]:
plot_avl_tree_iterative(avl)

## [12/12] Next Steps

In this lesson, we learned how to implement AVL trees, which are self-balancing binary search trees. We've implemented a method to add values and one to check if it contains a given value. We didn't implement deletions because it would significantly increase the lesson length. We encourage you to [read about](https://en.wikipedia.org/wiki/AVL_tree#Delete) it and try to implement it yourself.

Another common operation implemented in AVL trees is range queries. This kind of query retrieves all values in a given range.

We've implemented rotation operations to balance the tree and prevent the tree from growing too tall. It can be shown that the height of an AVL tree is at most **``2 × log(N)``**, where **``N``** is the number of nodes. If you're interested in a proof, you can find one [here](https://people.csail.mit.edu/alinush/6.006-spring-2014/avl-height-proof.pdf).
